# Load and Predict

We've now built our model, trained it, and saved it to file - now we can begin applying it to making predictions. First, we load the model with `tf.keras.models.load_model`.

In [2]:
import tensorflow as tf

model = tf.keras.models.load_model('sentiment_model')

# view model architecture to confirm we have save and loaded correctly
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
bert (Custom>TFBertMainLayer)   {'last_hidden_state' 108310272   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
dropout_37 (Dropout)            (None, 512, 768)     0           bert[0][0]                   

Before making our predictions we need to format our data, which requires two steps:

* Tokenizing the data using the `bert-base-cased` tokenizer.

* Transforming the data into a dictionary containing *'input_ids'* and *'attention_mask'* tensors.

In [3]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prep_data(text):
    tokens = tokenizer.encode_plus(text, max_length=512,
                                   truncation=True, padding='max_length',
                                   add_special_tokens=True, return_token_type_ids=False,
                                   return_tensors='tf')
    # tokenizer returns int32 tensors, we need to return float64, so we use tf.cast
    return {'input_ids': tf.cast(tokens['input_ids'], tf.float64),
            'attention_mask': tf.cast(tokens['attention_mask'], tf.float64)}

In [9]:
probs = model.predict(prep_data("hello world"))[0]

probs

array([0.0207338 , 0.05568625, 0.4257313 , 0.31074345, 0.18710516],
      dtype=float32)

In [11]:
import numpy as np

np.argmax(probs)

2

So we have made a test prediction, but we want to be applying this to real phrases from *test.tsv*. We will load the data into a dataframe, remove fragment duplicates based on *SentenceId*, then iterate through the list and create a new sentiment column.

In [14]:
import pandas as pd

# so we can see full phrase
pd.set_option('display.max_colwidth', None)

df = pd.read_csv('test.tsv', sep='\t')
df.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine effort .
1,156062,8545,An intermittently pleasing but mostly routine effort
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [15]:
df = df.drop_duplicates(subset=['SentenceId'], keep='first')
df.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine effort .
15,156076,8546,"Kidman is really the only thing that 's worth watching in Birthday Girl , a film by the stage-trained Jez Butterworth -LRB- Mojo -RRB- that serves as yet another example of the sad decline of British comedies in the post-Full Monty world ."
93,156154,8547,Once you get into its rhythm ... the movie becomes a heady experience .
117,156178,8548,I kept wishing I was watching a documentary about the wartime Navajos and what they accomplished instead of all this specious Hollywood hoo-ha .
158,156219,8549,"Kinnear does n't aim for our sympathy , but rather delivers a performance of striking skill and depth ."


Now we initialize our new sentiment column, and begin making predictions.

In [12]:
df['Sentiment'] = None

for i, row in df.iterrows():
    # get token tensors
    tokens = prep_data(row['Phrase'])
    # get probabilities
    probs = model.predict(tokens)
    # find argmax for winning class
    pred = np.argmax(probs)
    # add to dataframe
    df.at[i, 'Sentiment'] = pred

df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,156061,8545,An intermittently pleasing but mostly routine ...,3
15,156076,8546,Kidman is really the only thing that 's worth ...,3
93,156154,8547,Once you get into its rhythm ... the movie bec...,3
117,156178,8548,I kept wishing I was watching a documentary ab...,3
158,156219,8549,"Kinnear does n't aim for our sympathy , but ra...",3


In [16]:
df.tail()

,PhraseId,SentenceId,Phrase
66220,222281,11851,Not sweet enough to liven up its predictable story and will leave even fans of hip-hop sorely disappointed .
66239,222300,11852,"Nasty , ugly , pointless and depressing , even if you hate clowns ."
66253,222314,11853,"With tightly organized efficiency , numerous flashbacks and a constant edge of tension , Miller 's film is one of 2002 's involvingly adult surprises ."
66280,222341,11854,They should have called it Gutterball .
66287,222348,11855,"A long-winded , predictable scenario ."
